# Evolución del FIUBA map

En este caso, vamos a analizar cómo evoluciona la red del FIUBA map respecto a alumnos que alguna vez cursaron juntos.
El grafo que armaremos será de alumnos (nodos) que se conectan si alguna vez cursaron juntos una materia (aristas).

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import utils
import numpy as np
from scipy.optimize import curve_fit
from datetime import datetime

In [ ]:
df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_rel = df[['Padron', 'materia_id', 'materia_cuatrimestre']].copy()

# Sacamos a quienes no ordenan su carrera por cuatrimestres
df_rel.dropna(subset=['materia_cuatrimestre'], inplace=True)

df_simil = pd.merge(df_rel, df_rel, on=['materia_id', 'materia_cuatrimestre'])
df_simil = df_simil[df_simil['Padron_x'] != df_simil['Padron_y']]
df_simil = df_simil[['Padron_x', 'Padron_y', 'materia_cuatrimestre']]
df_simil = df_simil.sort_values(by=['materia_cuatrimestre'])
df_simil.head(3)

In [ ]:
# tomamos los cuatrimestres hasta la actualidad, sin tener en cuenta cuatrimestres que no pasaron porque la cantidad de nodos se mantiene constante
curr_year = datetime.now().year
curr_month = datetime.now().month
curr_cuatri = curr_year if curr_month < 3 else curr_year + 0.5
cuatris = df_simil[df_simil['materia_cuatrimestre'] < curr_cuatri]['materia_cuatrimestre'].unique()
cuatris

In [ ]:
graphs = []
stats = []
for cuatri in cuatris:
    df_simil_cuatri = df_simil[df_simil['materia_cuatrimestre'] <= cuatri]
    G = nx.from_pandas_edgelist(df_simil_cuatri, 
                                source='Padron_x', 
                                target='Padron_y',
                                create_using=nx.Graph())
    graphs.append((cuatri, G))
    stats.append([cuatri, len(G), len(G.edges), nx.diameter(G), f"{sum([n[1] for n in G.degree()]) / len(G):.2f}"])

stats = pd.DataFrame(stats, columns=['Cuatrimestre','Nodos','Aristas', 'Diámetro', 'Grado promedio'])
print(stats.to_string(index=False), end='\n\n')

fig, axs = plt.subplots(ncols=7, nrows=len(graphs)//7, figsize=(30,10))
ax = axs.flatten()
for i, g in enumerate(graphs):
    ax[i].set_title(g[0])
    nx.draw_networkx(g[1], pos=nx.kamada_kawai_layout(G), width=0.1 if i < 5 else 0.01, node_size=20, with_labels=False, ax=ax[i])

Lo primero que podemos notar es que, a lo largo del tiempo, las aristas crecen mucho más rápido que los nodos (ya veremos en qué proporción), y en consecuencia el diámetro se mantiene constante casi durante toda la evolución del grafo.

## Evolución macroscópica

A continuación analizaremos la relación entre la cantidad de nodos y la cantidad de aristas a lo largo del tiempo

In [ ]:
x = [len(g[1]) for g in graphs]
y = [len(g[1].edges) for g in graphs]

# Escala logaritmica
plt.xscale("log")
plt.yscale("log")
plt.xlim(10, 100000)
plt.ylim(10, 100000)
plt.xlabel("Cantidad de nodos")
plt.ylabel("Cantidad de aristas")

# Scatter plot
plt.scatter(x, y)

# Ecuación de la trendline
def myExpFunc(x, a, b):
    return a * np.power(x, b)

# Plot de la trendline
popt, pcov = curve_fit(myExpFunc, x, y)
newX = np.logspace(0, 3, base=10)
newY = myExpFunc(newX, *popt)
plt.plot(newX, newY, "r-")

# Tomo 2 puntos por los que pasa la recta de pendiente y calculo alfa
dy = y[11] - y[6]
dx = x[11] - x[6]

slope = (np.log10(dy)/np.log10(dx)).round(2)
print(f"Alpha es {slope}")

Sabemos que para la mayoría de las redes reales, la evolución de las aristas con respecto a la evolución de los nodos sigue la ley de potencia de densificación: $$ E(t) \propto N(t)^\alpha $$ donde $\alpha$ es el exponente de densificación en el rango de [1, 2]. En nuestro caso, $\alpha$ es mucho mayor a 1, lo que significa que a medida que evoluciona la red, el grado promedio de los nodos va vertiginosamente en aumento.

In [ ]:
x = [g[0] for g in graphs]
y = [nx.diameter(g[1]) for g in graphs]

plt.xlabel("Cuatrimestre")
plt.ylabel("Diámetro")
plt.plot(x, y, '-o')
plt.show()

Analicemos ahora el diámetro. En primer lugar, hay que decir que no es relevante lo que sucede previamente al 2017 porque la cantidad de nodos es muy chica. A partir de allí, se observa que el diámetro se mantiene casi constante en 4, salvo por el 2do cuatrimestre del 2019 en donde baja a 3. Una posible respuesta a esto es que, como vimos, la cantidad de aristas crece mucho más rápido que la cantidad de vértices, entonces hay más conexiones. Pero, si tuviéramos un grafo de Erdös-Rényi, el diámetro crecería igualmente, en contra de esta línea de pensamiento.

In [ ]:
degrees = [(g[0], sorted((d for n, d in g[1].degree()), reverse=True)) for g in graphs]

fig, axs = plt.subplots(ncols=7, nrows=len(graphs)//7, figsize=(30,10))
ax = axs.flatten()

for i, degree in enumerate(degrees):
    ax[i].set_xlabel("Grado del nodo")
    ax[i].set_ylabel("Cantidad de nodos")
    ax[i].set_title(degree[0])
    ax[i].bar(*np.unique(degree[1], return_counts=True)[::-1])

Si vemos cómo evoluciona la cantidad de nodos en relación al grado de los mismos, a partir de 2018.5, observamos que, si bien no se comporta exactamente como el modelo de Preferential Attachment, la mayoría de los nodos tienen un grado bajo y muy pocos tienen grado muy alto. A su vez, estos pocos con grado elevado siguen incrementándolo a lo largo del tiempo y explica, junto con la densificación, por qué el diámetro se mantiene constante a lo largo de la evolción.